### Reaction Score Distributions

Evaluate distributions of reaction scores and max theoretical score 

Identify the reactions that have the greatest SDs - virtually no differences in reaction scores across runs

In [1]:
from Compass.compass import models, compass, utils
import importlib
import pandas as pd
import os
import cplex
import numpy as np

In [71]:
# Load in model and initalize CPLEX problem 

model = models.init_model(model="HumanGEM", species="homo_sapiens", exchange_limit = 1.0, model_file_path='/home/dzhang/Compass_Integration/model_xml/0_1_models/MIAPaca_PAR_0_75.xml')

problem = compass.algorithm.initialize_cplex_problem(model)

In [72]:
# Reaction penalites 
reaction_penalties = pd.read_csv(
        '/home/dzhang/Compass_Integration/penalties.txt', sep="\t", header=0,
        usecols=[0, 1]) #0 is the Reaction column, only using 1st sampel 
reaction_penalties = reaction_penalties.set_index("Reaction").iloc[:, 0]
reaction_penalties

Reaction
MAR03905_pos    0.105728
MAR03907_pos    0.114925
MAR04097_pos    0.144783
MAR04099_pos    0.379900
MAR04108_pos    0.144783
                  ...   
MAR20045_pos    0.115123
MAR20052_pos    0.153752
MAR20066_pos    0.087496
MAR20069_pos    0.138128
MAR20070_pos    0.101348
Name: WT.d_S141_L007_R1_001, Length: 12494, dtype: float64

In [ ]:
# Trial run 

rxn_maxes = pd.DataFrame(columns = [f"Trial {num}" for num in range(100)])
rxn_maxes["Reaction ID"] = ""
rxn_maxes["Mean"] = ""
rxn_maxes["SD"] = ""
rxn_maxes = rxn_maxes.set_index("Reaction ID")
columns = rxn_maxes.columns.tolist()
columns = columns[-2:] + columns[:-2]
rxn_maxes = rxn_maxes[columns]

rxn_scores = pd.DataFrame(columns = [f"Trial {num}" for num in range(100)])
rxn_scores["Reaction ID"] = ""
rxn_scores["Mean"] = ""
rxn_scores["SD"] = ""
rxn_scores = rxn_scores.set_index("Reaction ID")
columns = rxn_scores.columns.tolist()
columns = columns[-2:] + columns[:-2]
rxn_scores = rxn_scores[columns]

### TAKES A LONG TIME, I stopped the run after ~2 hours

for reaction in reactions:
    
    for trial in range(100):
        partner_reaction = reaction.reverse_reaction

        # Set partner reaction upper-limit to 0 in problem
        # Store old limit for later to restore
        if partner_reaction is not None:
            partner_id = partner_reaction.id
            old_partner_ub = problem.variables.get_upper_bounds(partner_id)
            old_partner_lb = problem.variables.get_lower_bounds(partner_id)
            problem.variables.set_upper_bounds(partner_id, max(old_partner_lb, 0))

        utils.reset_objective(problem)
        problem.objective.set_linear(reaction.id, 1.0)
        problem.objective.set_name(str(reaction.id))
        problem.objective.set_sense(problem.objective.sense.maximize)

        problem.solve()

        rxn_max = problem.solution.get_objective_value()

        rxn_maxes.loc[reaction.id, f"Trial {trial}"] = rxn_max  

        if rxn_max != 0:
            problem.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=[reaction.id], val=[1.0])],
                senses=['R'],
                rhs=[0.95 * rxn_max],
                names=['REACTION_OPT'])
        if(problem.objective.get_name() != 'reaction_penalties'):
                utils.reset_objective(problem)
                problem.objective.set_linear(
                    list(reaction_penalties.iteritems())
                )
                problem.objective.set_name('reaction_penalties')
                problem.objective.set_sense(problem.objective.sense.minimize)
        problem.solve()
        try: 
            value = problem.solution.get_objective_value()
            # Insert values 
            rxn_scores.loc[reaction.id, f"Trial {trial}"] = value 
        except:
            rxn_scores.loc[reaction.id, f"Trial {trial}"] = -1 
        problem.linear_constraints.delete('REACTION_OPT')

        if partner_reaction is not None:
            partner_id = partner_reaction.id
            problem.variables.set_upper_bounds(partner_id, old_partner_ub)

    rxn_maxes.loc[reaction.id, "Mean"] = np.average(np.array(rxn_maxes.iloc[0]))
    rxn_maxes.loc[reaction.id, "SD"] = np.std(np.array(rxn_maxes.iloc[0]))

    rxn_scores.loc[reaction.id, "Mean"] = np.average(np.array(rxn_scores.iloc[0]))
    rxn_scores.loc[reaction.id, "SD"] = np.std(np.array(rxn_scores.iloc[0]))

In [41]:
rxn_maxes

,Trial 0,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5,Trial 6,Trial 7,Trial 8,Trial 9,...,Trial 90,Trial 91,Trial 92,Trial 93,Trial 94,Trial 95,Trial 96,Trial 97,Trial 98,Trial 99
Reaction ID,,,,,,,,,,,,,,,,,,,,,
MAR04097_pos,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,...,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312,371.895312


In [42]:
rxn_scores

,Trial 0,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5,Trial 6,Trial 7,Trial 8,Trial 9,...,Trial 90,Trial 91,Trial 92,Trial 93,Trial 94,Trial 95,Trial 96,Trial 97,Trial 98,Trial 99
Reaction ID,,,,,,,,,,,,,,,,,,,,,
MAR04097_pos,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,...,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353,4645.272353


In [44]:
np.std(np.array(rxn_maxes.iloc[0]))

1.2953632395913152e-11